In [ ]:
! pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# 크롬 옵션 설정 (헤드리스 모드)
chrome_options = Options()
chrome_options.add_argument('--headless')  # GUI를 표시하지 않음
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
from datetime import datetime

include = input('기사 제목에 포함시킬 내용을 입력하세요: ')

while True:
    start_date = input('시작 날짜를 입력하세요(예시: 20xx-xx-xx): ')
    end_date = input('종료 날짜를 입력하세요(예시: 20xx-xx-xx): ')
    
    try:
        start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
        end_date_obj = datetime.strptime(end_date, "%Y-%m-%d")
        
        if end_date_obj >= start_date_obj:
            break
        else:
            print("종료 날짜가 시작 날짜보다 빠를 수 없습니다. 다시 입력해주세요.")
    
    except ValueError:
        print("날짜 형식이 올바르지 않습니다. 다시 입력해주세요.")


start_date = list(map(int, start_date.split('-')))
end_date = list(map(int, end_date.split('-')))

In [ ]:
def select_date(start_date, end_date):
    global driver
    time.sleep(3)

    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{start_date[0]-1989}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[2]/div/div/div/ul/li[{start_date[1]}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[3]/div/div/div/ul/li[{start_date[2]}]/a').click()

    # 종료 날짜 클릭하기
    driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[1]/span[3]/a').click()

    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[1]/div/div/div/ul/li[{end_date[0]-1989}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[2]/div/div/div/ul/li[{end_date[1]}]/a').click()
    driver.find_element(By.XPATH, f'//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[2]/div[3]/div/div/div/ul/li[{end_date[2]}]/a').click()

    # 적용 클릭
    driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[3]/div/div[3]/div[3]/button').click()

In [ ]:
# 크롬 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

driver.get(url=f'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query={include}')

# 옵션 클릭
driver.find_element(By.XPATH, '//*[@id="snb"]/div[1]/div/div[2]/a').click()
driver.find_element(By.XPATH, '//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]').click()

select_date(start_date, end_date)

# 스크롤링을 위한 반복문
last_height = driver.execute_script("return document.body.scrollHeight")  # 현재 페이지의 전체 높이

while True:
    # 스크롤을 최하단으로 내리기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 로드 시간 대기

    # 새로운 페이지 높이를 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")

    # 새로운 페이지 높이가 기존 높이와 같으면, 더 이상 스크롤 불가능
    if new_height == last_height:
        break
    last_height = new_height  # 마지막 높이 갱신

press = [element.get_attribute("textContent") for element in driver.find_elements(By.CLASS_NAME, "info.press")] # 언론사
title = [element.get_attribute("title") for element in driver.find_elements(By.CLASS_NAME, "news_tit")] # 기사 제목
link = [element.get_attribute("href") for element in driver.find_elements(By.CLASS_NAME, "news_tit")]   # 기사 링크

for t, p, l in zip(title, press, link):
    print(f"언론사: {p} | 제목: {t} | 링크: {l}")

언론사: 뉴시스 | 제목: 박상우 장관 "국토지리정보원 50주년…공간정보 산업 선도" | 링크: https://www.newsis.com/view/NISX20241101_0002943258
언론사: 전자신문 | 제목: 서울디지털대 서동조 교수, 국토부 장관 표창 수상 | 링크: https://www.etnews.com/20241101000288
언론사: KBS | 제목: 국토지리정보원 창립 50주년 맞아 행사 개최 | 링크: https://news.kbs.co.kr/news/pc/view/view.do?ncd=8089342&ref=A
언론사: 이뉴스투데이 | 제목: '국토지리정보원 창립 50주년' 기념우표 나온다 | 링크: http://www.enewstoday.co.kr/news/articleView.html?idxno=2191878
언론사: 미디어펜 | 제목: [2024년 11월 1일 경제·산업 주요일정]금융위, 주간 업무 회의·국토부, 해외 건설 플랜트의 날·해수부, 수산양식박람회개회식 外 | 링크: https://www.mediapen.com/news/view/964900
언론사: 노컷뉴스 | 제목: 국토 높이 측정 정밀도 높인 '2024 국가 지오이드모델' 공개 | 링크: https://www.nocutnews.co.kr/news/6227764?utm_source=naver&utm_medium=article&utm_campaign=20241016060057
언론사: 아유경제 | 제목: [아유경제_부동산] 창립 50주년 맞은 국토지리정보원, 공간정보 미래 비전 선포 | 링크: http://www.areyou.co.kr/news/articleView.html?idxno=86246
언론사: 이데일리 | 제목: '고덕토평대로' 결정에 이수희 강동구청장 "'고덕대교' 사수"[동네방네] | 링크: http://www.edaily.co.kr/news/newspath.asp?newsid=04195126639058496
언론사: 문화일보 | 제목: 연천군, 고정밀 전

In [ ]:
import pandas as pd

# 데이터프레임 생성
df = pd.DataFrame({
    "언론사": press,
    "기사 제목": title,
    "링크": link
}).drop_duplicates(subset=['언론사','기사 제목']).reset_index(drop=True)

# 데이터프레임 출력
display(df)

,언론사,기사 제목,링크
0,뉴시스,"박상우 장관 ""국토지리정보원 50주년…공간정보 산업 선도""",https://www.newsis.com/view/NISX20241101_00029...
1,전자신문,"서울디지털대 서동조 교수, 국토부 장관 표창 수상",https://www.etnews.com/20241101000288
2,KBS,국토지리정보원 창립 50주년 맞아 행사 개최,https://news.kbs.co.kr/news/pc/view/view.do?nc...
3,이뉴스투데이,'국토지리정보원 창립 50주년' 기념우표 나온다,http://www.enewstoday.co.kr/news/articleView.h...
4,미디어펜,"[2024년 11월 1일 경제·산업 주요일정]금융위, 주간 업무 회의·국토부, 해외...",https://www.mediapen.com/news/view/964900
...,...,...,...
57,울산신문,"양산시, 고정밀 전자지도 구축사업 선정",https://www.ulsanpress.net/news/articleView.ht...
58,이코노뉴스,경기도 측량업체 지도점검 결과 58개소 적발…전년대비 39% 감소,http://www.econonews.co.kr/news/articleView.ht...
59,환경과조경,"더욱 정밀해진 지면 높이측량, ‘2024 국가 지오이드모델’ 공개",http://www.lak.co.kr/news/boardview.php?id=18498
60,뉴스경남,"양산시, 고정밀전자지도 구축 대상 지자체 선정",https://www.newsgn.com/news/articleView.html?i...


In [ ]:
df.to_csv('article_dataframe.csv', index=False, encoding='utf-8-sig')

from google.colab import files

# CSV 파일 다운로드
files.download('article_dataframe.csv')